In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data1=pd.read_csv(r"/kaggle/input/up-crime-data-year-2019/MR data - Compiled Data Set.csv")
data1

In [ ]:
print(data1.dtypes)

In [ ]:
# now first, we need to do preprocessing of this raw data
data1=data1.fillna(0)
data1.columns=np.arange(13)
data1

In [ ]:
data1=data1.drop([1,2], axis=1) # dropping unnecessary column(not really unnecessary but they are not involved in this numerical analysis)
c10=data1[10].copy()
data1

In [ ]:
dict3={"Property Disputes":1,"Family Dispute":2,"Petty Quarrels":3,"Money Disputes":4, "Personal Vendetta":5,"Love Affairs":6, "Casteism":7, "Unknown reasons":8,0:0}
data1[3]=data1[3].apply(lambda x: dict3[x]) # in column 3 , reasons are given in text, we convert them to integers(so that easy to count and looks more systematic)
data1.head(5)

In [ ]:
for i in range(0,len(data1)):
    if data1.iloc[i,8]!=0:
        print(data1.iloc[i,8]) # in column 8, getting the idea of what type of elements there are!!! 

In [ ]:
data1[13]=0
data1[14]=0  # creating two more columns so that we can split 10th columns(having elements like (8,10) or (2,3,6))
data1

In [ ]:
for i in range(0,len(data1)): # in column 10 there are elements like (2,3,6) or (10) or (7,10) , at max there are 3 integers separated by commas, so we put those integers in different columns to make our data easier to analyse
    if data1.iloc[i,8] not in [0,1,2,3,4,5,6,7,8,9,10]:
        str1=str(data1.iloc[i,8]).split(",")
        if len(str1) == 2: # this "if" condition will select elements like (7,10) or (8,10) or other triplets
            data1.iloc[i,8]=int(str1[0]) # index of column=8 , name =10
            data1.iloc[i,11]=int(str1[1]) # index of column=9 , name =11
            data1.iloc[i,12]=0 # index of column=10 
        elif len(str1)==3: # this "if" condition will select elements like (2,3,6) or (1,4,10) or other triplets
            data1.iloc[i,8]=int(str1[0]) # index of column=8 , column name =10
            data1.iloc[i,11]=int(str1[1]) # index of column=9 , name =11
            data1.iloc[i,12]=int(str1[2]) # index of column=10 , name =12
data1

In [ ]:
data1.describe()

In [ ]:
x=data1[3].value_counts() # we count , how many time a particular reason resulted into murder
type(x)
print(x)

In [ ]:
# pie chart showing major reasons for murders which are love affairs, petty quarrels and personal vendetta
counting=[10,16,19,14,16,30,2,40]
reasons_of_murders=["Property/Land Disputes","Family Dispute","Petty Quarrels","Money Disputes","Personal Vendetta","Love Affairs", "Casteism"," Unknown/other" ]
colors=["b","r","c","m","y","tab:pink","tab:olive","tab:gray"]
plt.pie(counting,labels=reasons_of_murders,colors=colors,startangle=90,shadow=True,explode=(0,0,0,0.1,0,0,0,0),autopct="%1.1f%%")
plt.title("pie chart")
plt.show  

In [ ]:
# murders
m1=pd.Series(data1[4]).sum() # Number_of_child_victims
m2=pd.Series(data1[5]).sum() # no. of male victims(adults)
m3=pd.Series(data1[6]).sum() # Number_of_female_victims(adult)
m=[m1,m2,m3]
victims_m=["child victims","adult male victims","adult female victims"]
plt.pie(m,labels=victims_m,colors=["tab:purple","tab:brown","tab:gray"],startangle=90,shadow=True,explode=(0,0,0),autopct="%1.1f%%")
plt.title("pie chart showing % of victims in murders")
plt.show
# this chart shows that in murders, most of the victims are adult male

In [ ]:
# kidnapping
k1=pd.Series(data1[7]).sum() # no. of child victims in kidnapping
k2=pd.Series(data1[8]).sum() # no. of adult male victims in kidnapping
k3=pd.Series(data1[9]).sum() # no of adult female victims in kidnapping 
k=[k1,k2,k3]
victims_k=["child victims","adult male victims","adult female victims"]
plt.pie(k,labels=victims_k,colors=["tab:purple","tab:brown","tab:gray"],startangle=90,shadow=True,explode=(0,0,0),autopct="%1.1f%%")
plt.title("pie chart showing % of victims in kidnapping")
plt.show
# this shows that most of the victims in kidnapping cases are children :(

In [ ]:
data1[10].value_counts()

In [ ]:
data1[13].value_counts()

In [ ]:
data1[14].value_counts()

In [ ]:
c10.value_counts()

In [ ]:
# let c7x= no. of times when crime 7 occured with crime x
c7=20 # 20 time , crime 7 happened alone
c710=26
c74=2
c79=2
c7others=4
c_7=[c7,c710,c74,c79,c7others]
along_with=["alone","crime10","crime4","crime9","others"]
plt.pie(c_7,labels=along_with,colors=["tab:purple","tab:brown","tab:gray","tab:orange","tab:blue"],startangle=90,shadow=True,explode=(.1,0,0,0,0),autopct="%1.1f%%")
plt.title("pie chart showing probability of  other crimes happening along with crime 7")
plt.show

# plot shows that when crime 7 happens, there is a probability of .37 that it will happen alone while .48 probability is that it will happen along with crime10 , which shows crime7 and crime10 have some sort of relation

In [ ]:
# now we find different no. of victims who suffered different crimes
v=np.array([0,0,0,0,0,0,0,0,0,0,0,0]) # v[i] will store the no. of people suffered from i-th crime
for i in range(0,len(data1)):
    v[int(data1.iloc[i,8])]+=int(data1.iloc[i,9])+int(data1.iloc[i,10])
    v[int(data1.iloc[i,11])]+=int(data1.iloc[i,9])+int(data1.iloc[i,10])
    v[int(data1.iloc[i,12])]+=int(data1.iloc[i,9])+int(data1.iloc[i,10])

    
print(v)

In [ ]:
plt.bar([1,2,3,4,5,6,7,8,9,10],[20,19,26,7,7,22,60,68,10,55],label="",color="b")
plt.title("bar graph")
plt.xlabel("crime no.")
plt.ylabel("no. of victims")
plt.show
# it shows that most of the victims suffered by crime 7,8 and 10 or we can say,  crime 7, 8 and 10 are affecting society on a way higher level 